# Playground

Like the name states, a notebook to play around with code based on learning from other notebooks

In [77]:
import os
from langchain.prompts  import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import tiktoken

Useful functions below (e.g. get a sense of token usage)

In [78]:
# Functions
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

In [68]:
llm_openai = OpenAI(model_name = "gpt-4-0613", temperature=0)


We'll be testing use of a prompt that requires answers to be based on quotes from One Piece. <b>Make sure to clean any pasted text prior to feeding into model to avoid errors </b>

Quotes source at: https://everydaypower.com/one-piece-quotes/

In [87]:
op_prompt = """You are an AI assistant that provides life advice based on the quotes below. 
The name after the dash is the author. Please reference quotes in answers. Include the author name. 
If you cannot answer based on the quotes, simply state that you don't know."

Quotes:
"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Everyone's weak when they hesitate." - Nami
"Everyone is expecting help from you, but you come here and don't think about doing anything to help those people?" - Nami
“I'm the smartest member in the Straw Hat group.” - Nami
“If you think I'm just another cute girl, you're dead wrong!” - Nami
“I'll have mapped out the entire world!” - Nami
“Just one more time and everything will be the same. Everything will come back to me. And I will be free from Arlong. Bellemere-san. Finally, I can laugh with all my heart.” - Nami

Question: {query}

Answer: """

In [88]:
op_prompt_template = PromptTemplate(
    input_variables=["query"],
    template=op_prompt
)

Test that the prompt works

In [89]:
print(op_prompt_template.format(query = "What should we do when we lose?"))

You are an AI assistant that provides life advice based on the quotes below. 
The name after the dash is the author. Please reference quotes in answers. Include the author name. 
If you cannot answer based on the quotes, simply state that you don't know."

Quotes:
"I've been picking fights all along!" - Luffy
"You are too from that generation." - Luffy
"Are we friends? Or are we foes? That kind of thing you decide yourselves!" - Luffy
“I don't wanna conquer anything. It's just that the person with the most freedom on the sea is the pirate king.” - Luffy
“If you don't take risks you can't create a future.” - Luffy
“Power isn't determined by your size, but by the size of your heart and dreams.” - Luffy
“Being alone is more painful than getting hurt.” - Luffy
“It's not about whether it's impossible or not, I'm doing it because I want to.” - Luffy
"Life is like a pencil that will surely run out, but will leave the beautiful writing of life." - Nami
"Everyone's weak when they hesitate." - N

Ensure we are within the token limit

In [90]:
print("Number of words: {}".format(
    len(op_prompt_template.format(query = "What should we do when we lose?"))
    ))

n_tokens = num_tokens_from_string(
    string=op_prompt_template.format(query = "What should we do when we lose?"),
    encoding_name="cl100k_base"
)

print("Number of tokens: {}".format(n_tokens))

Number of words: 1534
Number of tokens: 381


Feed a negative control ensuring LLM is not supposed what is not supposed to do according to prompt instructions

In [91]:
llm_openai(op_prompt_template.format(query = "What is Doge?"))

"I'm sorry, but none of the quotes provided give information about what Doge is."

Now feed it actual prompt of interest

In [92]:
print(
    llm_openai(
        op_prompt_template.format(query = "What should we do around weak people?")
        )
    )

We should encourage and support those who are weak. As Luffy said, "Power isn't determined by your size, but by the size of your heart and dreams." This means that even if someone appears weak, they can still have great strength within them. It's our job to help them realize that strength.
